In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "scoutu1234"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read({}))

df.drop(columns=['_id'], inplace=True)

df.sort_values(by="rec_num", ascending=True, inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png'
with open(image_filename, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

rescue_options = [
    {'label': 'Water', 'value': 'Water'},
    {'label': 'Mountain/Wilderness', 'value': 'Mountain'},
    {'label': 'Disaster/Individual Tracking', 'value': "Disaster"},
    {'label': 'Reset', 'value': "Reset"},
]

def filter_data(rescue_type):
    if rescue_type == 'Water':
        return df[(df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
                  (df['sex_upon_outcome'].str.contains('Intact Female', na=False)) &
                  (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif rescue_type == 'Mountain':
        return df[(df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky','Rottweiler'])) &
                  (df['sex_upon_outcome'].str.contains('Intact Male', na=False)) &
                  (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif rescue_type == 'Disaster':
        return df[(df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
                  (df['sex_upon_outcome'].str.contains('Intact Male', na=False)) &
                  (df['age_upon_outcome_in_weeks'].between(20, 300))]
    return df

app.layout = html.Div([
    html.Center(
        html.A(
            href="https://www.snhu.edu",
            target="_blank",
            children=[
                html.Img(
                    src=f"data:image/png;base64,{encoded_image}",
                    style={'width': '200px', 'border': '2px solid black'}
                )
            ]
        )
    ),

    html.Center(html.B(html.H1('SNHU CS-340 Dashboard modified by Sonny Coutu'))),
    html.Hr(),

    html.Div([
        html.Label('Select Rescue Type:'),
        dcc.RadioItems(id='rescue-filter', options=rescue_options, value='Reset', inline=True)
    ]),
    html.Br(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        filter_action="native",
        sort_action="native",
        style_cell={'textAlign': 'left'}
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'}, children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            children=[dcc.Graph(id='pie-chart')
        ]),
        html.Div(
            id='map-id',
            className='col s12 m6')
    ])
])

@app.callback(
    [Output('datatable-id', 'data'), Output('pie-chart', 'figure')],
    [Input('rescue-filter', 'value')]
)
def update_dashboard(rescue_type):
    filtered_df = filter_data(rescue_type)
    pie_chart = px.pie(filtered_df, names='sex_upon_outcome', title='Distribution by Sex') if not filtered_df.empty else px.pie(title='No Data Available')
    return filtered_df.to_dict('records'), pie_chart

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if not selected_rows:
        return []
    return [{
        'if': {'row_index': selected_rows[0]},
        'background_color': '#D2F3FF'
    }]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return [html.P("No data available to display on map")]
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty or len(dff.columns) < 17:
        return [html.P("Invalid data format, can't display map")]
    if not index or index[0] >= len(dff):
        return [html.P("Select a valid row")]
    row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[float(dff.iloc[row,13]), float(dff.iloc[row,14])],
                children=[
                    dl.Tooltip(dff.iloc[row, 4] if pd.notna(dff.iloc[row,4]) else "unknown"),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row, 9] if pd.notna(dff.iloc[row,9]) else "No name")
                    ])
                ]
            )
        ])
    ]

app.run_server(debug=True)


ServerSelectionTimeoutError: nv-desktop-services.apporto.com:34239: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 68bdca25958d3cbc4afc7378, topology_type: Unknown, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 34239) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:34239: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>